In [1]:
import torch 
from torch.autograd import Variable


In [2]:
x = Variable(torch.ones((2,2)), requires_grad=True)
y = x + 1 

print("x: ", x)
print("y: ", y)

x:  tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
y:  tensor([[2., 2.],
        [2., 2.]], grad_fn=<AddBackward0>)


In [3]:
y.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

In [4]:
# simple gradient
a = Variable(torch.FloatTensor([2, 3]), requires_grad=True)
b = a + 3
c = b * b * 3
out = c.mean()

print("input:")
print(a)
print("ouput:")
print(out)

input:
tensor([2., 3.], requires_grad=True)
ouput:
tensor(91.5000, grad_fn=<MeanBackward1>)


In [5]:
out.backward()

In [6]:
print("input gradients: ")
print(a.grad)

input gradients: 
tensor([15., 18.])


对于非标量的情况

In [14]:
m = Variable(torch.FloatTensor([[2, 3]]), requires_grad=True)
n = Variable(torch.zeros(1, 2))
n[0, 0] = m[0, 0] ** 2
n[0, 1] = m[0, 1] ** 3

In [15]:
print("m: ")
print(m)
print("n: ")
print(n)

m: 
tensor([[2., 3.]], requires_grad=True)
n: 
tensor([[ 4., 27.]], grad_fn=<CopySlices>)


In [16]:
m.data

tensor([[2., 3.]])

In [11]:
n.backward(m.data)

In [12]:
m.grad

tensor([[ 8., 81.]])

In [17]:
n.backward(torch.FloatTensor([[1,1]]))

In [18]:
m.grad

tensor([[ 4., 27.]])

$m=(x_1=2, x_2=3 ), k=(x_1^2 + 3*x_2, x_2^2 + 2*x_1)$

In [20]:
m = Variable(torch.FloatTensor([[2, 3]]), requires_grad=True)
j = torch.zeros(2 ,2)
k = Variable(torch.zeros(1, 2))

k[0, 0] = m[0, 0] ** 2 + 3 * m[0 ,1]
k[0, 1] = m[0, 1] ** 2 + 2 * m[0, 0]

In [21]:
k.backward(torch.FloatTensor([[1,1]]))

In [22]:
m.grad

tensor([[6., 9.]])

In [23]:
k

tensor([[13., 13.]], grad_fn=<CopySlices>)

```k.backward(parameters)```接受的参数parameters必须要和k的大小一模一样，然后作为k的系数传回去，什么意思呢，我们通过上面的例子来解释这个问题你就知道了。

我们已经知道我们得到的k=(k1,k2)，以及传入的参数是1和1，那么是如何得到这6和9这两个结果的呢？

其实第一个结果是通过$1×\frac{dk_1}{dx1}+1×\frac{dk_2}{dx_1}=2x_1+2=6$这样得到的，是不是有点理解这个操作是怎么完成的了，我们再来看看第二个结果，$1 × \frac{dk_1}{dx_2}+1×\frac{dk_2}{dx_2}=3+2x_2=9$，这样我们就得到了这两个结果，原来我们传入的参数是每次求导的一个系数。

In [27]:
# jacobian
m = Variable(torch.FloatTensor([[2, 3]]), requires_grad=True)
j = torch.zeros(2 ,2)
k = Variable(torch.zeros(1, 2))

k[0, 0] = m[0, 0] ** 2 + 3 * m[0 ,1]
k[0, 1] = m[0, 1] ** 2 + 2 * m[0, 0]
k.backward(torch.FloatTensor([[1, 0]]),retain_graph=True)
j[:, 0] = m.grad.data
m.grad.data.zero_()
k.backward(torch.FloatTensor([[0, 1]]))
j[:, 1] = m.grad.data
print('jacobian matrix is')
print(j)

jacobian matrix is
tensor([[4., 2.],
        [3., 6.]])
